In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/KISTI

/content/drive/MyDrive/KISTI


# 1. 폴더 재구성 및 train/test spilit

In [ ]:
# 동일한 split 조건에서 실험하기 위해 seed를 고정

import numpy as np
import random
import os

# 파일/디렉토리 복사, 이동을 위한 라이브러리
import shutil

seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)

## 1.1. '선종 구분' Task를 위한 폴더 및 파일 구성

In [ ]:
# shutil.copytree(src, dst) : src -> dst, src 디렉토리의 파일들을 dst 디렉토리로 복사
# src : e.g., "dataset_origial/1.국내어선/1. 낚시어선/이동"
# dst : e.g., "dataset_shiptype/train/fishing-boat"
# dirs_exist_ok : dst 폴더가 이미 존재하면, 신규 폴더를 생성하지 않고 데이터만 복사

In [ ]:
def shiptype_data_split():

  # 읽는 폴더 경로 정보
  read_root_path = "dataset_original"
  read_countries_path = ["/1. 국내어선", "/2. 중국어선"]
  read_statuses_path = ["/이동", "/조업", "/표류"]

  # 생성 폴더 경로 정보
  root_path = "dataset_shiptype"
  train_path = "/train"
  test_path = "/test"
  shiptypes_korea = {"/1. 낚시어선" : "/fishing-boat",
                     "/2. 저인망" : "/trawling",
                     "/3. 채낚기" : "/fishing",
                     "/4. 연승" : "/longline",
                     "/5. 통발" : "/trap",
                     "/6. 안강망": "/angang-mang"}
  shiptypes_china = {"/1. 타망": "/ta-mang",
                     "/2. 유망": "/yu-mang",
                     "/3. 범장망": "/beomjang-mang",
                     "/4. 등광조망": "/light-mang"}


  # 한국어선 데이터 복사
  for read_path, copy_path in shiptypes_korea.items():

    # 이동, 조업, 표류
    for status in read_statuses_path:
      src_path = read_root_path + read_countries_path[0] + read_path + status
      dst_path = root_path + train_path + copy_path
      shutil.copytree(src_path, dst_path, dirs_exist_ok=True)

    # 10%를 random 하게 골라서 test 폴더로 이동
    src_path = root_path + train_path + copy_path
    dst_path = root_path + test_path + copy_path
    os.makedirs(dst_path)

    imgs = glob(src_path + "/*")
    num_test_imgs = int(len(imgs) * 0.1)
    randomly_selected_test_imgs = np.random.choice(imgs, num_test_imgs, replace=False)

    for img in randomly_selected_test_imgs:
      shutil.move(img, dst_path)

    print("# of train/test for", copy_path[1:] ,":", len(glob(src_path + "/*")), "/", len(glob(dst_path + "/*")))


  # 중국어선 데이터 복사
  for read_path, copy_path in shiptypes_china.items():

    # 이동, 조업, 표류
    for status in read_statuses_path:
      src_path = read_root_path + read_countries_path[1] + read_path + status
      dst_path = root_path + train_path + copy_path
      shutil.copytree(src_path, dst_path, dirs_exist_ok=True)

    # 10%를 random 하게 골라서 test 폴더로 이동
    src_path = root_path + train_path + copy_path
    dst_path = root_path + test_path + copy_path
    os.makedirs(dst_path)

    imgs = glob(src_path + "/*")
    num_test_imgs = int(len(imgs) * 0.1)
    randomly_selected_test_imgs = np.random.choice(imgs, num_test_imgs, replace=False)

    for img in randomly_selected_test_imgs:
      shutil.move(img, dst_path)

    print("# of train/test for", copy_path[1:] ,":", len(glob(src_path + "/*")), "/", len(glob(dst_path + "/*")))

## 1.2. '조업상태' 구분 Task를 위한 폴더 및 파일 구성

In [ ]:
# shutil.copytree(src, dst) : src -> dst, src 디렉토리의 파일들을 dst 디렉토리로 복사
# src : e.g., "dataset_origial/1.국내어선/1. 낚시어선/이동"
# dst : e.g., "dataset_shiptype/train/fishing-boat"
# dirs_exist_ok : dst 폴더가 이미 존재하면, 신규 폴더를 생성하지 않고 데이터만 복사

In [ ]:
def status_data_split():

  # 읽는 폴더 경로 정보
  read_root_path = "dataset_original"
  read_countries_path = ["/1. 국내어선", "/2. 중국어선"]
  shiptypes_korea = ["/1. 낚시어선", "/2. 저인망", "/3. 채낚기", "/4. 연승", "/5. 통발", "/6. 안강망"]
  shiptypes_china = ["/1. 타망", "/2. 유망", "/3. 범장망", "/4. 등광조망"]

  # 생성 폴더 경로 정보
  root_path = "dataset_status"
  train_path = "/train"
  test_path = "/test"
  statuses_path = {"/이동":"/moving",
                   "/조업":"/operating",
                   "/표류":"/drifting"}

  # 한국어선 데이터 복사
  for shiptype in shiptypes_korea:

    # 이동, 조업, 표류 copy
    for read_path, copy_path in statuses_path.items():
      src_path = read_root_path + read_countries_path[0] + shiptype + read_path
      dst_path = root_path + train_path + copy_path
      shutil.copytree(src_path, dst_path, dirs_exist_ok=True)

  # 중국어선 데이터 복사
  for shiptype in shiptypes_china:

    # 이동, 조업, 표류 copy
    for read_path, copy_path in statuses_path.items():
      src_path = read_root_path + read_countries_path[1] + shiptype + read_path
      dst_path = root_path + train_path + copy_path
      shutil.copytree(src_path, dst_path, dirs_exist_ok=True)



  # 이동, 조업, 표류 별로 10%를 random 하게 골라서 test 폴더로 이동
  for status in statuses_path.values():
    src_path = root_path + train_path + status
    dst_path = root_path + test_path + status
    os.makedirs(dst_path)

    imgs = glob(src_path + "/*")
    num_test_imgs = int(len(imgs) * 0.1)
    randomly_selected_test_imgs = np.random.choice(imgs, num_test_imgs, replace=False)

    for img in randomly_selected_test_imgs:
      shutil.move(img, dst_path)

    print("# of train/test for", status[1:] ,":", len(glob(src_path + "/*")), "/", len(glob(dst_path + "/*")))

In [ ]:
# 실행은 1~2분 가량 소요
# 실제로 구글드라이브 폴더에서 이동된 파일들이 보이는 것까지는 한 3~4분
shiptype_data_split()
status_data_split()

# of train/test for fishing-boat : 115 / 12
# of train/test for trawling : 73 / 8
# of train/test for fishing : 55 / 6
# of train/test for longline : 51 / 5
# of train/test for trap : 68 / 7
# of train/test for angang-mang : 46 / 5
# of train/test for ta-mang : 376 / 41
# of train/test for yu-mang : 177 / 19
# of train/test for beomjang-mang : 393 / 43
# of train/test for light-mang : 87 / 9
# of train/test for moving : 518 / 57
# of train/test for operating : 309 / 34
# of train/test for driftling : 611 / 67
